## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-07-21-23-53-17 +0000,world,Trump administration releases MLK Jr.'s FBI re...,https://www.cbc.ca/news/world/trump-releases-f...
1,2025-07-21-22-58-25 +0000,world,Ex-police officer sentenced to 33 months for v...,https://www.cbc.ca/news/world/breonna-taylor-h...
2,2025-07-21-21-20-37 +0000,world,Trump is serious about wanting Washington Comm...,https://www.cbc.ca/sports/football/nfl/donald-...
3,2025-07-21-19-57-20 +0000,world,Father of Montreal girl who was found dead in ...,https://www.cbc.ca/news/canada/montreal/missin...
4,2025-07-21-18-40-07 +0000,world,The Cosby Show star Malcolm-Jamal Warner dead ...,https://www.cbc.ca/news/entertainment/malcolm-...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2301/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
0,trump,2
11,police,2
26,house,2
32,dead,2
2,releases,1


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
6,2025-07-21-11-12-30 +0000,world,Japan PM Ishiba vows to stay on after party lo...,https://www.cbc.ca/news/world/japan-house-elec...,12
7,2025-07-21-02-59-47 +0000,world,"Police say driver shot, attacked by crowd in L...",https://www.cbc.ca/news/world/fernando-ramirez...,12
0,2025-07-21-23-53-17 +0000,world,Trump administration releases MLK Jr.'s FBI re...,https://www.cbc.ca/news/world/trump-releases-f...,11
2,2025-07-21-21-20-37 +0000,world,Trump is serious about wanting Washington Comm...,https://www.cbc.ca/sports/football/nfl/donald-...,11
3,2025-07-21-19-57-20 +0000,world,Father of Montreal girl who was found dead in ...,https://www.cbc.ca/news/canada/montreal/missin...,11


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
6,12,2025-07-21-11-12-30 +0000,world,Japan PM Ishiba vows to stay on after party lo...,https://www.cbc.ca/news/world/japan-house-elec...
7,12,2025-07-21-02-59-47 +0000,world,"Police say driver shot, attacked by crowd in L...",https://www.cbc.ca/news/world/fernando-ramirez...
0,11,2025-07-21-23-53-17 +0000,world,Trump administration releases MLK Jr.'s FBI re...,https://www.cbc.ca/news/world/trump-releases-f...
3,11,2025-07-21-19-57-20 +0000,world,Father of Montreal girl who was found dead in ...,https://www.cbc.ca/news/canada/montreal/missin...
5,9,2025-07-21-12-18-11 +0000,world,At least 19 killed after Bangladesh air force ...,https://www.cbc.ca/news/world/bangladesh-air-f...
1,8,2025-07-21-22-58-25 +0000,world,Ex-police officer sentenced to 33 months for v...,https://www.cbc.ca/news/world/breonna-taylor-h...
4,8,2025-07-21-18-40-07 +0000,world,The Cosby Show star Malcolm-Jamal Warner dead ...,https://www.cbc.ca/news/entertainment/malcolm-...
2,7,2025-07-21-21-20-37 +0000,world,Trump is serious about wanting Washington Comm...,https://www.cbc.ca/sports/football/nfl/donald-...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
